In [0]:
# Notebook: 02_Final_Quality_Checks.ipynb

processo_nome = 'final_data_testing'
status_final = 'sucesso'
detalhes_log = 'Todos os testes de integridade concluídos.'
violations_ref = 0
violations_const = 0

try:
    spark.sql(f"""
    INSERT INTO pl_delivery_analysis.log_processamento (processo, etapa, status, timestamp, detalhes)
    VALUES ('{processo_nome}', 'inicio', 'executando', CURRENT_TIMESTAMP(), 'Iniciando testes de integridade e constantes');
    """)

    # --- TESTE 1: INTEGRIDADE REFERENCIAL (store_id) ---
    print("Executando Teste de Integridade Referencial (store_id)...")
    
    # Objetivo: Contar store_id na Fato (Gold) que NÃO existem na Dimensão (Gold)
    df_ref_integrity = spark.sql("""
    SELECT 
        COUNT(*) AS violation_count
    FROM pl_delivery_analysis.tbl_fato_delivery_gold fact
    LEFT ANTI JOIN pl_delivery_analysis.tbl_dim_store_gold dim
        ON fact.store_id = dim.store_id
    """)
    violations_ref = df_ref_integrity.collect()[0]['violation_count']

    # --- TESTE 2: VALIDAÇÃO DE CONSTANTES DA UE ---
    print("Executando Teste de Validação de Constantes da UE...")

    # Objetivo: Recalcular a UE para uma amostra e comparar com o valor armazenado.
    # Se houver diferença, as constantes ou a lógica de cálculo foram alteradas.
    df_const_test = spark.sql("""
    WITH constantes AS (
        SELECT 
            0.18 AS comissao_plataforma,
            0.70 AS repasse_entregador,
            0.02 AS taxa_transacao
    ),
    recalculo AS (
        SELECT
            p.order_id,
            p.lucro_bruto_unitario AS stored_ue,
            (
                (p.subtotal_bruto * c.comissao_plataforma) + (p.delivery_fee_cliente * (1 - c.repasse_entregador)) 
                - (p.delivery_fee_cliente * c.repasse_entregador) 
                - (p.subtotal_bruto * c.taxa_transacao)
            ) AS calculated_ue
        FROM pl_delivery_analysis.tbl_fact_pedidos_silver p
        CROSS JOIN constantes c
        LIMIT 10000 -- Amostra rápida para verificar a lógica/constantes
    )
    SELECT COUNT(*) AS violation_count
    FROM recalculo
    WHERE ROUND(stored_ue, 4) != ROUND(calculated_ue, 4);
    """)
    violations_const = df_const_test.collect()[0]['violation_count']

    # --- LOGICA DE VERIFICAÇÃO FINAL ---
    if violations_ref > 0 or violations_const > 0:
        status_final = 'alerta'
        detalhes_log = f"ALERTA DQ: Falhas de Integridade Referencial ({violations_ref}) ou Inconsistências de Constantes ({violations_const})."
        print(f"ALERTA DQ: {detalhes_log}")
    else:
        detalhes_log = "Testes de Integridade Referencial e Validação de Constantes passaram com sucesso."
        
    # --- REGISTRAR MÉTRICAS DE TESTE ---
    spark.sql(f"""
    INSERT INTO pl_delivery_analysis.metricas_qualidade (metrica, valor, data_calculo)
    VALUES 
    ('test_ref_integrity_violations', {violations_ref}, CURRENT_TIMESTAMP()),
    ('test_const_validation_violations', {violations_const}, CURRENT_TIMESTAMP());
    """)


except Exception as e:
    status_final = 'falha'
    erro_msg = str(e).replace("'", "") 
    detalhes_log = f"ERRO CRÍTICO no Data Testing: {erro_msg}"
    print(f"ERRO CRÍTICO: {detalhes_log}")
    raise e

finally:
    # --- LOG FINAL ---
    spark.sql(f"""
    INSERT INTO pl_delivery_analysis.log_processamento (processo, etapa, status, timestamp, detalhes)
    VALUES ('{processo_nome}', 'fim', '{status_final}', CURRENT_TIMESTAMP(), '{detalhes_log}');
    """)

In [0]:
%sql
SELECT 
    metrica,
    valor,
    data_calculo
FROM 
    pl_delivery_analysis.metricas_qualidade
WHERE 
    metrica LIKE 'test_%'
ORDER BY data_calculo DESC;